## Analiza danych w zadanym formacie

dim, ts_name,     best_value,                 steps_to_best_value

np.

188, PPO_G2_130k, 158810.53550787075, 18135

Set-up

In [14]:
# 1. IMPORTY I KONFIGURACJA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# opcjonalnie: zwiększenie rozmiaru wykresów
plt.rcParams['figure.figsize'] = (10, 6)

# 2. Wczytanie danych
df = pd.read_csv("G2_130K_TestsResult.csv")

# 3. Krótkie „smoking‑gun”
display(df.head())
display(df.describe(include='all'))

,dim,ts_name,best_value,steps_to_best_value
0,188,PPO_G2_130k,158810.535508,18135
1,188,Linear,158746.585746,22945
2,188,ReciprocalV1,156991.399427,3902
3,188,ReciprocalV2,157353.582348,722
4,188,GeometricV1,160202.271191,40581


,dim,ts_name,best_value,steps_to_best_value
count,44528.000000,44528,44528.000000,44528.000000
unique,NaN,8,NaN,NaN
top,NaN,PPO_G2_130k,NaN,NaN
freq,NaN,5566,NaN,NaN
mean,175.318541,NaN,166930.439338,24397.991421
std,72.691366,NaN,123588.008542,14627.258775
min,50.000000,NaN,8659.923305,0.000000
25%,112.000000,NaN,55471.119739,11710.000000
50%,175.000000,NaN,139982.173345,23776.500000
75%,238.000000,NaN,264802.230346,36913.250000


In [15]:
#Średnia, odchylenie standardowe, min i max dla 'best_value' i 'steps_to_best_value' w grupach ts_name
group_stats = df.groupby('ts_name')[['best_value', 'steps_to_best_value']].agg(['mean', 'std', 'min', 'max'])

# wykonanie różnicy a.x - PPO.x
baseline = group_stats.loc['PPO_G2_130k']
diff_stats = group_stats.subtract(baseline, axis='columns').divide(baseline, axis='columns').multiply(100,axis='columns')

# Wyświetlenie wyników
styled = diff_stats.style\
    .background_gradient(subset=[('best_value','mean'), ('steps_to_best_value','mean')], cmap='viridis')\
    .format("{:.4f}")

# Aby zobaczyć ją w notebooku:
styled

tabela przedstawia polepszenie lub pogorszenie w % względem wybranego typu sterowania

(x.mean - PPO.mean)/PPO.mean * 100

dodatnia wartość >> PPO_G2 bylo lepsze

ujemna wartość >> PPO_G2 bylo gorsze